# Kaggle Project-1
# Brain Tumor MRI Classification
- 출처: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
- 목표: ImageDataGenerator와 TL(transfer learning)을 이용해서 과대적합을 최소화하여 90%이상의 정확도 달성
  - 만약 목표를 달성하지 못한다면, 문제점과 해결 방안 모색

## Dataset

In [2]:
# 구글 마운트
# Google mount
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# 현재 디렉토리 /content로 이미지 데이터 복사 후 현재 디렉토리 ROOT_DIR로 명명
# Copy image data to /content and name /content as ROOT_DIR
import shutil
shutil.copy(os.path.join("/content/drive/MyDrive/Colab Notebooks/", "brain_tumor.zip"), "/content")
ROOT_DIR = "/content"

In [4]:
# brain_tumor 디렉토리 생성과 동시에 압축해제 후 brain_tumor 디렉토리 DATA_ROOT_DIR로 명명
# Decompress data in brain_tumor dir and name brain_tumor dir as DATA_ROOT_DIR
import zipfile
with zipfile.ZipFile(os.path.join(ROOT_DIR, "brain_tumor.zip"), "r") as target_file:
  target_file.extractall("/content/brain_tumor")
  DATA_ROOT_DIR = "/content/brain_tumor"

## Path naming

In [5]:
# brain_tumor 디렉토리에 Testing과 Training 디렉토리 확인 후
# 각각 TEST_DATA_ROOT_DIR, TRIAN_DATA_ROOT_DIR로 명명
# Name Testing dir and Training dir in brain_tumor dir as TEST_DATA_ROOT_DIR, TRAIN_DATA_ROOT_DIR
TRIAN_DATA_ROOT_DIR = "/content/brain_tumor/Training"
TEST_DATA_ROOT_DIR = "/content/brain_tumor/Testing"

## Checking total number of data and labels

In [9]:
# number of Training data
import os
train_label_name_list = os.listdir(TRIAN_DATA_ROOT_DIR)
print(train_label_name_list)

for train_label_name in train_label_name_list:
  print(train_label_name, "=>", len(os.listdir(os.path.join(TRIAN_DATA_ROOT_DIR, train_label_name))))

['pituitary_tumor', 'no_tumor', 'meningioma_tumor', 'glioma_tumor']
pituitary_tumor => 827
no_tumor => 395
meningioma_tumor => 822
glioma_tumor => 826


In [10]:
# number of Testing data
test_label_name_list = os.listdir(TEST_DATA_ROOT_DIR)
print(test_label_name_list)

for test_label_name in test_label_name_list:
  print(test_label_name, "=>", len(os.listdir(os.path.join(TEST_DATA_ROOT_DIR, test_label_name))))

['pituitary_tumor', 'no_tumor', 'meningioma_tumor', 'glioma_tumor']
pituitary_tumor => 74
no_tumor => 105
meningioma_tumor => 115
glioma_tumor => 100


## Setting image shape

In [12]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

## Importing data using ImageDataGenerator